In [1]:
import pandas as pd
from transformers import pipeline
from transformers import AutoModelForSequenceClassification
import torch
from torch.utils.data import DataLoader
from transformers import Trainer, TrainingArguments,AutoTokenizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

In [ ]:
# Load the dataset
data = pd.read_csv('resources/diabetes_df.csv')

# Display the first few rows of the dataset
data.head()

In [ ]:
# Define model
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

In [4]:
# Define features and labels
features = data.drop(columns=['Diabetes_binary'])  # Adjust if your target column is named differently
labels = data['Diabetes_binary']

# Convert to DataFrame for easier handling
features_df = pd.DataFrame(features, columns=features.columns)

In [19]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    features_df.values.tolist(),  # Convert DataFrame to list of lists
    labels.values.tolist(),
    test_size=0.2,
    random_state=42)

In [20]:
class DiabetesDataset(torch.utils.data.Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __getitem__(self, idx):
        item = { 'input_ids': torch.tensor(self.features[idx], dtype=torch.float) }
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

In [22]:
train_dataset = DiabetesDataset(train_texts, train_labels)
val_dataset = DiabetesDataset(val_texts, val_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

In [2]:
pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-1B")

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.2-1B.
403 Client Error. (Request ID: Root=1-6716f2b4-373a60f622d9e2440a202d7e;d9cfc7c0-5144-4949-9191-6538a460737b)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-1B/resolve/main/config.json.
Your request to access model meta-llama/Llama-3.2-1B is awaiting a review from the repo authors.